# Feathr Feature Store on Azure Demo Notebook

This notebook illustrates the use of Feature Store to create a model that predicts NYC Taxi fares. It includes these steps:


This tutorial demonstrates the key capabilities of Feathr, including:

1. Install and set up Feathr with Azure
2. Create shareable features with Feathr feature definition configs.
3. Create a training dataset via point-in-time feature join.
4. Compute and write features.
5. Train a model using these features to predict fares.
6. Materialize feature value to online store.
7. Fetch feature value in real-time from online store for online scoring.

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The feature flow is as below:

![Feature Flow](https://github.com/linkedin/feathr/blob/main/docs/images/feature_flow.png?raw=true)

## Prerequisite: Use Quick Start Template to Provision Azure Resources

Feathr has native cloud integration. To use Feathr on Azure, you only need three steps:

- Get the `Principal ID` of your account by running `az ad signed-in-user show --query objectId -o tsv` in the link below (Select "Bash" if asked), and write down that value (something like `b65ef2e0-42b8-44a7-9b55-abbccddeefff`). Think this ID as something representing you when accessing Azure, and it will be used to grant permissions in the next step in the UI.

[Launch Cloud Shell](https://shell.azure.com/bash)

- Click the button below to deploy a minimal set of Feathr resources for demo purpose. You will need to fill in the `Principal ID` and `Resource Prefix`. You will need "Owner" permission of the selected subscription.

[![Deploy to Azure](https://aka.ms/deploytoazurebutton)](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Flinkedin%2Ffeathr%2Fmain%2Fdocs%2Fhow-to-guides%2Fazure_resource_provision.json)

- Run the cells below.

And the architecture is as below. In the above template, we are using Synapse as Spark provider, use Azure Data Lake Gen2 as offline store, and use Redis as online store, Azure Purview (Apache Atlas compatible) as feature reigstry. 


![Architecture](https://github.com/linkedin/feathr/blob/main/docs/images/architecture.png?raw=true)

In [0]:
! pip install feathr

Requirement already satisfied: feathr in /databricks/python3/lib/python3.8/site-packages (0.3.2)
Requirement already satisfied: py4j in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.10.9.3)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from feathr) (4.64.0)
Requirement already satisfied: pyyaml in /databricks/python3/lib/python3.8/site-packages (from feathr) (6.0)
Requirement already satisfied: pyarrow in /databricks/python3/lib/python3.8/site-packages (from feathr) (4.0.0)
Requirement already satisfied: pandavro in /databricks/python3/lib/python3.8/site-packages (from feathr) (1.6.0)
Requirement already satisfied: python-snappy in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.6.1)
Requirement already satisfied: Jinja2 in /databricks/python3/lib/python3.8/site-packages (from feathr) (2.11.3)
Requirement already satisfied: azure-keyvault-secrets in /databricks/python3/lib/python3.8/site-packages (from feathr) (4.4.0)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from feathr) (2.25.1)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from feathr) (1.2.4)
Requirement already satisfied: graphlib-backport in /databricks/python3/lib/python3.8/site-packages (from feathr) (1.0.3)
Requirement already satisfied: loguru in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.6.0)
Requirement already satisfied: google>=3.0.0 in /databricks/python3/lib/python3.8/site-packages (from feathr) (3.0.0)
Requirement already satisfied: azure-storage-file-datalake>=12.5.0 in /databricks/python3/lib/python3.8/site-packages (from feathr) (12.6.0)
Requirement already satisfied: Click in /databricks/python3/lib/python3.8/site-packages (from feathr) (8.1.2)
Requirement already satisfied: pyspark>=3.1.2 in /databricks/python3/lib/python3.8/site-packages (from feathr) (3.2.1)
Requirement already satisfied: redis in /databricks/python3/lib/python3.8/site-packages (from feathr) (4.2.2)
Requirement already satisfied: pyhocon in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.3.59)
Requirement already satisfied: azure-identity in /databricks/python3/lib/python3.8/site-packages (from feathr) (1.9.0)
Requirement already satisfied: deltalake in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.5.6)
Requirement already satisfied: pyapacheatlas in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.11.0)
Requirement already satisfied: azure-synapse-spark in /databricks/python3/lib/python3.8/site-packages (from feathr) (0.7.0)
Requirement already satisfied: google-api-python-client>=2.41.0 in /databricks/python3/lib/python3.8/site-packages (from feathr) (2.45.0)
Requirement already satisfied: azure-storage-blob<13.0.0,>=12.10.0 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-file-datalake>=12.5.0->feathr) (12.11.0)
Requirement already satisfied: azure-core<2.0.0,>=1.15.0 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-file-datalake>=12.5.0->feathr) (1.23.1)
Requirement already satisfied: msrest>=0.6.21 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-file-datalake>=12.5.0->feathr) (0.6.21)
Requirement already satisfied: typing-extensions>=4.0.1 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake>=12.5.0->feathr) (4.2.0)
Requirement already satisfied: six>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.15.0->azure-storage-file-datalake>=12.5.0->feathr) (1.15.0)
Requirement already satisfied: cryptography>=2.1.4 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-blob<13.0.0,>=12.10.0->azure-storage-file-datalake>=12.5.0->feathr) (36.0.2)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from c

In [0]:
! pip install pandavro scikit-learn 

Requirement already satisfied: pandavro in /databricks/python3/lib/python3.8/site-packages (1.6.0)
Requirement already satisfied: scikit-learn in /databricks/python3/lib/python3.8/site-packages (0.24.1)
Requirement already satisfied: pandas>=1.1.5 in /databricks/python3/lib/python3.8/site-packages (from pandavro) (1.2.4)
Requirement already satisfied: fastavro>=0.14.11 in /databricks/python3/lib/python3.8/site-packages (from pandavro) (1.4.10)
Requirement already satisfied: six>=1.9 in /databricks/python3/lib/python3.8/site-packages (from pandavro) (1.15.0)
Requirement already satisfied: numpy>=1.7.0 in /databricks/python3/lib/python3.8/site-packages (from pandavro) (1.19.2)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.1.5->pandavro) (2020.5)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.1.5->pandavro) (2.8.1)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.8/site-packages (from scikit-learn) (1.0.1)
Requirement already satisfied: scipy>=0.19.1 in /databricks/python3/lib/python3.8/site-packages (from scikit-learn) (1.6.2)
Requirement already satisfied: threadpoolctl>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from scikit-learn) (2.1.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

Login to Azure with a device code (You will see instructions in the output):

In [0]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import json
import requests

Get all the required credentials from Azure KeyValut

In [0]:
# Get current databricks notebook context
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host_name = ctx.tags().get("browserHostName").get()
host_token = ctx.apiToken().get()
cluster_id = ctx.tags().get("clusterId").get()

# Get redis credentials; This is to parse Redis connection string.
redis_port=""
redis_host=""
redis_password=""
redis_ssl=""

# Set the resource link
os.environ['online_store__redis__host'] = redis_host
os.environ['online_store__redis__port'] = redis_port
os.environ['online_store__redis__ssl_enabled'] = redis_ssl
os.environ['REDIS_PASSWORD']=redis_password


feathr_runtime_location = "https://azurefeathrstorage.blob.core.windows.net/public/feathr-assembly-LATEST.jar"

databricks_config = {'run_name':'','existing_cluster_id':cluster_id,'libraries':[{'jar':''}],'spark_jar_task':{'main_class_name':'','parameters':['']}}
os.environ['spark_config__databricks__workspace_instance_url'] = "https://" + host_name
os.environ['spark_config__databricks__config_template']='{"run_name":"FEATHR_FILL_IN","new_cluster":{"spark_version":"10.4.x-scala2.12","node_type_id":"Standard_D3_v2","num_workers":2,"spark_conf":{"FEATHR_FILL_IN":"FEATHR_FILL_IN"}},"libraries":[{"jar":"FEATHR_FILL_IN"}],"spark_jar_task":{"main_class_name":"FEATHR_FILL_IN","parameters":["FEATHR_FILL_IN"]}}'
os.environ['spark_config__databricks__work_dir']='dbfs:/feathr_getting_started'
os.environ['spark_config__databricks__feathr_runtime_location']=feathr_runtime_location
os.environ['project_config__project_name']='feathr_getting_started'
os.environ['DATABRICKS_WORKSPACE_TOKEN_VALUE'] = host_token

## Prerequisite: Configure the required environment (Don't need to update if using the above Quick Start Template)

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. If you use Feathr CLI to create a workspace, you should have a folder with a file called `feathr_config.yaml` in it with all the required configurations. Otherwise, update the configuration below.

The code below will write this configuration string to a temporary location and load it to Feathr. Please still refer to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It should also have more explanations on the meaning of each variable.

In [0]:
import tempfile
yaml_config = """
# Please refer to https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml for explanations on the meaning of each field.
api_version: 1
project_config:
  project_name: 'feathr_getting_started2'
  required_environment_variables:
    - 'REDIS_PASSWORD'
    - 'AZURE_CLIENT_ID'
    - 'AZURE_TENANT_ID'
    - 'AZURE_CLIENT_SECRET'
offline_store:
  adls:
    adls_enabled: true
  wasb:
    wasb_enabled: true
  s3:
    s3_enabled: false
    s3_endpoint: 's3.amazonaws.com'
  jdbc:
    jdbc_enabled: false
    jdbc_database: 'feathrtestdb'
    jdbc_table: 'feathrtesttable'
  snowflake:
    url: "dqllago-ol19457.snowflakecomputing.com"
    user: "feathrintegration"
    role: "ACCOUNTADMIN"
spark_config:
  # choice for spark runtime. Currently support: azure_synapse, databricks
  # The `databricks` configs will be ignored if `azure_synapse` is set and vice versa.
  spark_cluster: "databricks"
  spark_result_output_parts: "1"

online_store:
  redis:
    host: 'feathrazuretest3redis.redis.cache.windows.net'
    port: 6380
    ssl_enabled: True
feature_registry:
  purview:
    type_system_initialization: true
    purview_name: 'feathrazuretest3-purview1'
    delimiter: '__'
"""
tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)


## Setup necessary environment variables (Skip if using the above Quick Start Template)

You should setup the environment variables in order to run this sample. More environment variables can be set by referring to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It also has more explanations on the meaning of each variable.

In [0]:
# os.environ['REDIS_PASSWORD'] = ''
# os.environ['AZURE_CLIENT_ID'] = ''
# os.environ['AZURE_TENANT_ID'] = ''
# os.environ['AZURE_CLIENT_SECRET'] = ''

# # Optional envs if you are using different runtimes
# os.environ['DATABRICKS_WORKSPACE_TOKEN_VALUE'] = ''

# Initialize Feathr Client

In [0]:
client = FeathrClient(config_path=tmp.name)

2022-04-20 04:51:15.729 | INFO | feathr._envvariableutil:get_environment_variable:61 - REDIS_PASSWORD is not set in the environment variables.

## View the data

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The data is as below

In [0]:
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/green_tripdata_2020-04_with_index.csv")

/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
 has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Out[8]:

,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,0,2.0,2020-04-01 00:44:02,2020-04-01 00:52:23,N,1.0,42,41,1.0,1.68,8.00,0.5,0.5,0.0,0.00,NaN,0.3,9.30,1.0,1.0,0.0
1,1,2.0,2020-04-01 00:24:39,2020-04-01 00:33:06,N,1.0,244,247,2.0,1.94,9.00,0.5,0.5,0.0,0.00,NaN,0.3,10.30,2.0,1.0,0.0
2,2,2.0,2020-04-01 00:45:06,2020-04-01 00:51:13,N,1.0,244,243,3.0,1.00,6.50,0.5,0.5,0.0,0.00,NaN,0.3,7.80,2.0,1.0,0.0
3,3,2.0,2020-04-01 00:45:06,2020-04-01 01:04:39,N,1.0,244,243,2.0,2.81,12.00,0.5,0.5,0.0,0.00,NaN,0.3,13.30,2.0,1.0,0.0
4,4,2.0,2020-04-01 00:00:23,2020-04-01 00:16:13,N,1.0,75,169,1.0,6.79,21.00,0.5,0.5,0.0,0.00,NaN,0.3,22.30,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35607,35607,NaN,2020-04-30 23:29:00,2020-04-30 23:57:00,NaN,NaN,37,147,NaN,11.41,35.82,0.0,0.0,0.0,6.12,NaN,0.3,42.24,NaN,NaN,NaN
35608,35608,NaN,2020-04-30 23:11:00,2020-04-30 23:47:00,NaN,NaN,188,230,NaN,11.17,35.45,0.0,0.0,0.0,0.00,NaN,0.3,38.50,NaN,NaN,NaN
35609,35609,NaN,2020-04-30 23:18:00,2020-04-30 23:46:00,NaN,NaN,205,37,NaN,14.37,34.54,0.0,0.0,0.0,0.00,NaN,0.3,34.84,NaN,NaN,NaN
35610,35610,NaN,2020-04-30 23:55:00,2020-05-01 00:10:00,NaN,NaN,37,188,NaN,4.25,16.72,0.0,0.0,0.0,0.00,NaN,0.3,17.02,NaN,NaN,NaN


## Defining Features with Feathr

In Feathr, a feature is viewed as a function, mapping from entity id or key, and timestamp to a feature value. For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://github.com/linkedin/feathr/blob/main/docs/concepts/feature-definition.md)


1. The typed key (a.k.a. entity id) identifies the subject of feature, e.g. a user id, 123.
2. The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
3. The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.

Note that, in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.

### Define Sources Section with UDFs
A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

In [0]:
from pyspark.sql import SparkSession, DataFrame
def feathr_udf_day_calc(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import dayofweek, dayofyear, col
    df = df.withColumn("fare_amount_cents", col("fare_amount")*100)
    return df

batch_source = HdfsSource(name="nycTaxiBatchSource",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/green_tripdata_2020-04_with_index.csv",
                          event_timestamp_column="lpep_dropoff_datetime",
                          preprocessing=feathr_udf_day_calc,
                          timestamp_format="yyyy-MM-dd HH:mm:ss")

### Define Anchors and Features
A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

In [0]:
f_trip_distance = Feature(name="f_trip_distance",
                          feature_type=FLOAT, transform="trip_distance")

features = [
    f_trip_distance,
    Feature(name="f_is_long_trip_distance",
            feature_type=BOOLEAN,
            transform="cast_float(trip_distance)>30"),
    Feature(name="f_day_of_week",
            feature_type=INT32,
            transform="dayofweek(lpep_dropoff_datetime)"),
]

request_anchor = FeatureAnchor(name="request_features",
                               source=INPUT_CONTEXT,
                               features=features)

### Window aggregation features

For window aggregation features, see the supported fields below:

Note that the `agg_func` should be any of these:

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |


After you have defined features and sources, bring them together to build an anchor:


Note that if the data source is from the observation data, the `source` section should be `INPUT_CONTEXT` to indicate the source of those defined anchors.

In [0]:
location_id = TypedKey(key_column="DOLocationID",
                       key_column_type=ValueType.INT32,
                       description="location id in NYC",
                       full_name="nyc_taxi.location_id")
agg_features = [Feature(name="f_location_avg_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="AVG",
                                                          window="90d")),
                Feature(name="f_location_max_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="MAX",
                                                          window="90d")),
                Feature(name="f_location_total_fare_cents",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="fare_amount_cents",
                                                          agg_func="SUM",
                                                          window="90d")),
                ]

agg_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=batch_source,
                           features=agg_features)

### Derived Features Section
Derived features are the features that are computed from other features. They could be computed from anchored features, or other derived features.

In [0]:

f_trip_distance_rounded = DerivedFeature(name="f_trip_distance_rounded",
                                     feature_type=INT32,
                                     input_features=[f_trip_distance],
                                     transform="f_trip_distance * 10")


And then we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features (which is not anchored to a source).

In [0]:
client.build_features(anchor_list=[agg_anchor, request_anchor], derived_feature_list=[
                       f_trip_distance_rounded])

## Create training data using point-in-time correct feature join

A training dataset usually contains entity id columns, multiple feature columns, event timestamp column and label/target column. 

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. 

To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/linkedin/feathr/blob/main/docs/concepts/point-in-time-join.md)

In [0]:

output_path = 'dbfs:/feathrazure_test.parquet'


feature_query = FeatureQuery(
    feature_list=["f_location_avg_fare", "f_trip_distance_rounded", "f_is_long_trip_distance", "f_location_total_fare_cents"], key=location_id)
settings = ObservationSettings(
    observation_path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/green_tripdata_2020-04_with_index.csv",
    event_timestamp_column="lpep_dropoff_datetime",
    timestamp_format="yyyy-MM-dd HH:mm:ss")
client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path=output_path,
                            execution_configuratons={"spark.feathr.outputFormat": "parquet"}
                           )
client.wait_job_to_finish(timeout_sec=500)

2022-04-20 04:51:18.073 | DEBUG | feathr._databricks_submission:upload_file:113 - /tmp/tmpwttnn5le/feathr_pyspark_driver.py is uploaded to location: dbfs:/feathr_getting_started/feathr_pyspark_driver.py
2022-04-20 04:51:18.780 | DEBUG | feathr._databricks_submission:upload_file:113 - /tmp/tmpwttnn5le/feature_join_conf/feature_join.conf is uploaded to location: dbfs:/feathr_getting_started/feature_join.conf
2022-04-20 04:51:18.781 | INFO | feathr._databricks_submission:upload_or_get_cloud_path:90 - Uploading folder /tmp/tmpwttnn5le/feature_conf/
2022-04-20 04:51:19.368 | DEBUG | feathr._databricks_submission:upload_file:113 - /tmp/tmpwttnn5le/feature_conf/auto_generated_anchored_features.conf is uploaded to location: dbfs:/feathr_getting_started/auto_generated_anchored_features.conf
2022-04-20 04:51:20.285 | DEBUG | feathr._databricks_submission:upload_file:113 - /tmp/tmpwttnn5le/feature_conf/auto_generated_request_features.conf is uploaded to location: dbfs:/feathr_getting_started/auto_generated_request_features.conf
2022-04-20 04:51:20.861 | DEBUG | feathr._databricks_submission:upload_file:113 - /tmp/tmpwttnn5le/feature_conf/auto_generated_derived_features.conf is uploaded to location: dbfs:/feathr_getting_started/auto_generated_derived_features.conf
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - S3_ACCESS_KEY is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - S3_SECRET_KEY is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - ADLS_ACCOUNT is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - ADLS_KEY is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - BLOB_ACCOUNT is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - BLOB_KEY is not set in the environment variables.
2022-04-20 04:51:20.863 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_TABLE is not set in the environment variables.
2022-04-20 04:51:20.864 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_USER is not set in the environment variables.
2022-04-20 04:51:20.864 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_PASSWORD is not set in the environment variables.
2022-04-20 04:51:20.864 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_DRIVER is not set in the environment variables.
2022-04-20 04:51:20.864 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_AUTH_FLAG is not set in the environment variables.
2022-04-20 04:51:20.864 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_TOKEN is not set in the environment variables.
2022-04-20 04:51:20.878 | INFO | feathr._envvariableutil:get_environment_variable:61 - JDBC_SF_PASSWORD is not set in the environment variables.
2022-04-20 04:51:30.949 | DEBUG | feathr._databricks_submission:upload_or_get_cloud_path:80 - https://azurefeathrstorage.blob.core.windows.net/public/feathr-assembly-LATEST.jar is downloaded and then uploaded to location: dbfs:/feathr_getting_started/feathr-assembly-LATEST.jar
2022-04-20 04:51:30.957 | DEBUG | feathr._databricks_submission:upload_or_get_cloud_path:84 - Skipping file dbfs:/feathr_getting_started/feathr_pyspark_driver.py as it is already in the cloud
2022-04-20 04:51:31.464 | INFO | feathr._databricks_submission:submit_feathr_job:172 - Feathr Job Submitted Sucessfully. View more details here: https://adb-2474129336842816.16.azuredatabricks.net/?o=2474129336842816#job/200732889873337/run/63018
2022-04-20 04:51:31.767 | DEBUG | feathr._databricks_submission:wait_for_completion:184 - Current Spark job status: PENDING
2022-04-20 04:52:01.960 | DEBUG | feathr._databricks_submiss

## Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [0]:
from feathr.job_utils import get_result_df
df_res = get_result_df(client)

2022-04-20 05:01:25.451 | DEBUG | feathr._databricks_submission:wait_for_completion:184 - Current Spark job status: SUCCESS
Downloading result files: 0%| | 0/148 [00:00<?, ?it/s]Downloading result files: 1%|▏ | 2/148 [00:00<00:36, 3.98it/s]Downloading result files: 3%|▎ | 4/148 [00:01<00:48, 2.95it/s]Downloading result files: 3%|▎ | 5/148 [00:02<01:12, 1.97it/s]Downloading result files: 4%|▍ | 6/148 [00:02<01:19, 1.78it/s]Downloading result files: 5%|▍ | 7/148 [00:03<01:22, 1.71it/s]Downloading result files: 5%|▌ | 8/148 [00:04<01:17, 1.80it/s]Downloading result files: 6%|▌ | 9/148 [00:04<01:12, 1.92it/s]Downloading result files: 7%|▋ | 10/148 [00:04<01:11, 1.94it/s]Downloading result files: 7%|▋ | 11/148 [00:05<01:08, 1.99it/s]Downloading result files: 8%|▊ | 12/148 [00:05<01:06, 2.05it/s]Downloading result files: 9%|▉ | 13/148 [00:06<01:04, 2.10it/s]Downloading result files: 9%|▉ | 14/148 [00:06<01:09, 1.94it/s]Downloading result files: 10%|█ | 15/148 [00:07<01:08, 1.93it/s]Downloading result files: 11%|█ | 16/148 [00:07<01:05, 2.00it/s]Downloading result files: 11%|█▏ | 17/148 [00:08<01:04, 2.02it/s]Downloading result files: 12%|█▏ | 18/148 [00:08<01:04, 2.00it/s]Downloading result files: 13%|█▎ | 19/148 [00:09<01:14, 1.74it/s]Downloading result files: 14%|█▎ | 20/148 [00:10<01:12, 1.77it/s]Downloading result files: 14%|█▍ | 21/148 [00:10<01:10, 1.81it/s]Downloading result files: 15%|█▍ | 22/148 [00:11<01:07, 1.86it/s]Downloading result files: 16%|█▌ | 23/148 [00:11<01:04, 1.93it/s]Downloading result files: 16%|█▌ | 24/148 [00:12<01:03, 1.95it/s]Downloading result files: 17%|█▋ | 25/148 [00:12<01:09, 1.78it/s]Downloading result files: 18%|█▊ | 26/148 [00:13<01:05, 1.85it/s]Downloading result files: 18%|█▊ | 27/148 [00:13<01:01, 1.96it/s]Downloading result files: 19%|█▉ | 28/148 [00:14<00:59, 2.01it/s]Downloading result files: 20%|█▉ | 29/148 [00:14<00:57, 2.05it/s]Downloading result files: 20%|██ | 30/148 [00:15<00:56, 2.09it/s]Downloading result files: 21%|██ | 31/148 [00:15<00:56, 2.07it/s]Downloading result files: 22%|██▏ | 32/148 [00:16<00:55, 2.08it/s]Downloading result files: 22%|██▏ | 33/148 [00:16<00:54, 2.11it/s]Downloading result files: 23%|██▎ | 34/148 [00:17<00:53, 2.14it/s]Downloading result files: 24%|██▎ | 35/148 [00:17<00:52, 2.14it/s]Downloading result files: 24%|██▍ | 36/148 [00:18<00:51, 2.17it/s]Downloading result files: 25%|██▌ | 37/148 [00:18<00:56, 1.97it/s]Downloading result files: 26%|██▌ | 38/148 [00:19<00:54, 2.02it/s]Downloading result files: 26%|██▋ | 39/148 [00:19<00:53, 2.05it/s]Downloading result files: 27%|██▋ | 40/148 [00:20<01:00, 1.78it/s]Downloading result files: 28%|██▊ | 41/148 [00:20<01:02, 1.70it/s]Downloading result files: 28%|██▊ | 42/148 [00:21<00:59, 1.78it/s]Downloading result files: 29%|██▉ | 43/148 [00:21<00:56, 1.86it/s]Downloading result files: 30%|██▉ | 44/148 [00:22<00:55, 1.88it/s]Downloading result files: 30%|███ | 45/148 [00:22<00:52, 1.95it/s]Downloading result files: 31%|███ | 46/148 [00:23<00:52, 1.96it/s]Downloading result files: 32%|███▏ | 47/148 [00:23<00:53, 1.90it/s]Downloading result files: 32%|███▏ | 48/148 [00:24<00:51, 1.94it/s]Downloading result files: 33%|███▎ | 49/148 [00:24<00:50, 1.95it/s]Downloading result files: 34%|███▍ | 50/148 [00:25<00:49, 1.98it/s]Downloading result files: 34%|███▍ | 51/148 [00:25<00:47, 2.04it/s]Downloading result files: 35%|███▌ | 52/148 [00:26<00:51, 1.85it/s]Downloading result files: 36%|███▌ | 53/148 [00:27<00:50, 1.88it/s]Downloading result files: 36%|███▋ | 54/148 [00:27<00:48, 1.93it/s]Downloading result files: 37%|███▋ | 55/148 [00:28<00:47, 1.97it/s]Downloading result files: 38%|███▊ | 56/148 [00:28<00:45, 2.02it/s]Downloading result files: 39%|███▊ | 57/148 [00:29<00:44, 2.04it/s]Downloading result files: 39%|███▉ | 58/148 [00:29<00:42, 2.10it/s]Downloading result files: 40%|███▉ | 59/148 [00:29<00:41, 2.13it/s]Downloading result files: 41%|████ | 60/148 [00:30

In [0]:
df_res

Out[19]:

,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,f_is_long_trip_distance,f_location_total_fare_cents,f_location_avg_fare,f_trip_distance_rounded
0,24650,None,2020-04-01 09:42:00,2020-04-01 09:52:00,None,None,121,73,None,2.12,8.0,0.0,0.0,0.0,0.0,None,0.3,8.3,None,None,None,False,800.0,8.000000,21
1,4123,2.0,2020-04-04 20:14:06,2020-04-04 20:34:30,N,1.0,260,73,1.0,7.13,23.5,0.5,0.5,0.0,0.0,None,0.3,24.8,2.0,1.0,0.0,False,3150.0,15.750000,71
2,26545,None,2020-04-07 09:15:00,2020-04-07 09:32:00,None,None,28,73,None,3.9,15.24,0.0,0.0,0.0,0.0,None,0.3,15.54,None,None,None,False,4674.0,15.579999,39
3,7259,2.0,2020-04-09 01:01:59,2020-04-09 01:16:36,N,1.0,92,73,1.0,1.82,11.5,0.5,0.5,0.0,0.0,None,0.3,12.8,2.0,1.0,0.0,False,5824.0,14.559999,18
4,12532,2.0,2020-04-16 01:22:45,2020-04-16 01:30:33,N,1.0,92,73,1.0,2.18,8.5,0.5,0.5,1.96,0.0,None,0.3,11.76,1.0,1.0,0.0,False,6674.0,13.348000,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35607,33897,None,2020-04-28 16:27:00,2020-04-28 17:01:00,None,None,29,219,None,15.28,44.54,0.0,0.0,0.0,0.0,None,0.3,44.84,None,None,None,False,252801.0,35.605774,152
35608,34240,None,2020-04-29 07:00:00,2020-04-29 07:09:00,None,None,130,219,None,5.07,13.77,0.0,0.0,2.75,0.0,None,0.3,16.82,None,None,None,False,254178.0,35.302502,50
35609,34521,None,2020-04-29 15:59:00,2020-04-29 16:32:00,None,None,197,219,None,15.04,22.42,0.0,0.0,2.75,0.0,None,0.3,25.47,None,None,None,False,256420.0,35.126026,150
35610,35156,None,2020-04-30 13:13:00,2020-04-30 13:48:00,None,None,228,219,None,23.15,45.55,0.0,0.0,2.75,0.0,None,0.3,48.6,None,None,None,False,260975.0,35.266891,231


## Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [0]:
# remove columns
from sklearn.ensemble import GradientBoostingRegressor
final_df = df_res
final_df.drop(["lpep_pickup_datetime", "lpep_dropoff_datetime",
              "store_and_fwd_flag"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['fare_amount'] = final_df['fare_amount'].astype("float64")


train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["fare_amount"], axis=1),
                                                    final_df["fare_amount"],
                                                    test_size=0.2,
                                                    random_state=42)
model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


Model MAPE:
0.02640633467797197

Model Accuracy:
0.9735936653220281

## Materialize feature value into offline/online storage

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:

In [0]:
backfill_time = BackfillTime(start=datetime(
    2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))
redisSink = RedisSink(table_name="nycTaxiDemoFeature")
settings = MaterializationSettings("nycTaxiTable",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["f_location_avg_fare", "f_location_max_fare"])

client.materialize_features(settings)
client.wait_job_to_finish(timeout_sec=500)


We can then get the features from the online store (Redis):

## Fetching feature value for online inference

For features that are already materialized by the previous step, their latest value can be queried via the client's
`get_online_features` or `multi_get_online_features` API.

In [0]:
res = client.get_online_features('nycTaxiDemoFeature', '265', [
                                 'f_location_avg_fare', 'f_location_max_fare'])

In [0]:
client.multi_get_online_features("nycTaxiDemoFeature", ["239", "265"], [
                                 'f_location_avg_fare', 'f_location_max_fare'])
